In [2]:
import pandas as pd
from tqdm import tqdm

In [20]:
import pandas as pd
from tqdm import tqdm

file_path = "ol_dump_ratings_2025-02-11.txt"
output_file = "openlibrary_ratings.csv"

data_list = []

import pandas as pd

file_path = "ol_dump_ratings_2025-02-11.txt"
output_file = "openlibrary_ratings.csv"

column_names = ["Work_ID", "Book_ID", "Rating", "Date"]

df = pd.read_csv(file_path, sep="\t", header=None, names=column_names, dtype=str)

df["Rating"] = pd.to_numeric(df["Rating"], errors="coerce")

df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

df.to_csv(output_file, index=False)

print(f"File saved as {output_file}")

File saved as openlibrary_ratings.csv


In [21]:
df


,Work_ID,Book_ID,Rating,Date
0,/works/OL17882343W,NaN,3,2018-06-20
1,/works/OL1629179W,/books/OL22981670M,5,2018-06-20
2,/works/OL4226036W,/books/OL10690412M,5,2018-06-20
3,/works/OL5264255W,/books/OL2719185M,5,2018-06-20
4,/works/OL1681415W,/books/OL2582724M,5,2018-06-20
...,...,...,...,...
518673,/works/OL461939W,/books/OL8494161M,3,2022-07-30
518674,/works/OL1708091W,/books/OL10691251M,3,2022-07-30
518675,/works/OL8299102W,/books/OL7695147M,4,2022-07-30
518676,/works/OL15191772W,/books/OL24324060M,4,2022-07-30


In [17]:
df

,Work_ID,Book_ID,Rating,Timestamp
0,OL17882343W,NaN,NaN,3
1,OL1629179W,OL22981670M,5.0,2018-06-20
2,OL4226036W,OL10690412M,5.0,2018-06-20
3,OL5264255W,OL2719185M,5.0,2018-06-20
4,OL1681415W,OL2582724M,5.0,2018-06-20
...,...,...,...,...
518673,OL461939W,OL8494161M,3.0,2022-07-30
518674,OL1708091W,OL10691251M,3.0,2022-07-30
518675,OL8299102W,OL7695147M,4.0,2022-07-30
518676,OL15191772W,OL24324060M,4.0,2022-07-30


Count the number of unique works and books in the rating dataset.

In [22]:
# summarize the rating data for each work
work_ratings = df.groupby("Work_ID")["Rating"].agg(["count", "mean", "std"]).reset_index()

In [23]:
work_isbn_counts = df['Work_ID'].nunique()
work_isbn_counts

246044

In [27]:
df.count()

Work_ID    518678
Book_ID    466164
Rating     518678
Date       518678
dtype: int64

In [24]:
work_ratings.to_csv("work_ratings_summary.csv", index=False)

In [26]:
work_ratings

,Work_ID,count,mean,std
0,/works/OL10000000W,1,3.0,NaN
1,/works/OL100001W,1,1.0,NaN
2,/works/OL1000035W,1,4.0,NaN
3,/works/OL1000043W,1,3.0,NaN
4,/works/OL100004W,1,3.0,NaN
...,...,...,...,...
246039,/works/OL99997W,1,4.0,NaN
246040,/works/OL999981W,5,3.4,1.67332
246041,/works/OL999982W,2,5.0,0.00000
246042,/works/OL999983W,1,5.0,NaN
